In [4]:
"""
Script de récupération de documents géopolitiques pour le TP RAG
Utilise des sources RSS publiques et gratuites
"""

import feedparser
import requests
import json
import os
from datetime import datetime
from bs4 import BeautifulSoup
import time

# Créer un dossier pour stocker les documents
DOCS_FOLDER = "documents_geopolitique"
if not os.path.exists(DOCS_FOLDER):
    os.makedirs(DOCS_FOLDER)

# Sources RSS d'actualité géopolitique (gratuites et accessibles)
SOURCES_RSS = {
    "monde_diplo": {
        "url": "https://www.monde-diplomatique.fr/recents.xml",
        "name": "Le Monde Diplomatique",
        "lang": "fr"
    },
    "france24_fr": {
        "url": "https://www.france24.com/fr/rss",
        "name": "France 24",
        "lang": "fr"
    },
    "rfi_monde": {
        "url": "https://www.rfi.fr/fr/monde/rss",
        "name": "RFI Monde",
        "lang": "fr"
    },
    "un_news": {
        "url": "https://news.un.org/feed/subscribe/en/news/all/rss.xml",
        "name": "UN News",
        "lang": "en"
    },
    "reuters_world": {
        "url": "https://www.reutersagency.com/feed/?taxonomy=best-regions&post_type=best",
        "name": "Reuters World",
        "lang": "en"
    }
}

def nettoyer_texte(html_text):
    """Nettoie le HTML pour extraire le texte brut"""
    if not html_text:
        return ""
    soup = BeautifulSoup(html_text, 'html.parser')
    return soup.get_text().strip()

def recuperer_articles_rss(source_key, source_info, max_articles=20):
    """Récupère les articles d'une source RSS"""
    print(f"\nRécupération depuis {source_info['name']}...")
    
    try:
        feed = feedparser.parse(source_info['url'])
        articles = []
        
        for i, entry in enumerate(feed.entries[:max_articles]):
            article = {
                "source": source_info['name'],
                "langue": source_info['lang'],
                "titre": entry.get('title', 'Sans titre'),
                "date": entry.get('published', ''),
                "lien": entry.get('link', ''),
                "resume": nettoyer_texte(entry.get('summary', '')),
                "contenu": nettoyer_texte(entry.get('content', [{}])[0].get('value', '')) if 'content' in entry else ""
            }
            
            # Si pas de contenu, utiliser le résumé
            if not article['contenu']:
                article['contenu'] = article['resume']
            
            # Ne garder que les articles avec du contenu
            if article['contenu'] and len(article['contenu']) > 100:
                articles.append(article)
                print(f"  ✓ Article {i+1}: {article['titre'][:60]}...")
        
        return articles
    
    except Exception as e:
        print(f"  ✗ Erreur: {e}")
        return []

def sauvegarder_documents(tous_articles):
    """Sauvegarde les documents dans des fichiers JSON et TXT"""
    
    # Sauvegarder tous les articles dans un JSON
    filename_json = os.path.join(DOCS_FOLDER, f"articles_geopolitique_{datetime.now().strftime('%Y%m%d')}.json")
    with open(filename_json, 'w', encoding='utf-8') as f:
        json.dump(tous_articles, f, ensure_ascii=False, indent=2)
    print(f"\n✓ {len(tous_articles)} articles sauvegardés dans {filename_json}")
    
    # Créer aussi des fichiers texte individuels pour chaque article
    txt_folder = os.path.join(DOCS_FOLDER, "articles_txt")
    if not os.path.exists(txt_folder):
        os.makedirs(txt_folder)
    
    for i, article in enumerate(tous_articles):
        filename_txt = os.path.join(txt_folder, f"article_{i+1:03d}.txt")
        with open(filename_txt, 'w', encoding='utf-8') as f:
            f.write(f"TITRE: {article['titre']}\n")
            f.write(f"SOURCE: {article['source']}\n")
            f.write(f"DATE: {article['date']}\n")
            f.write(f"LANGUE: {article['langue']}\n")
            f.write(f"\nCONTENU:\n{article['contenu']}\n")
    
    print(f"✓ Articles individuels sauvegardés dans {txt_folder}")
    
    return filename_json

def generer_corpus_thematique():
    """Génère aussi quelques documents thématiques de référence"""
    
    documents_reference = [
        {
            "titre": "Les grandes puissances mondiales en 2025",
            "contenu": """Les États-Unis maintiennent leur position de première puissance mondiale malgré la montée de la Chine. 
            L'Union européenne cherche son autonomie stratégique. La Russie reste un acteur majeur dans les questions énergétiques et militaires. 
            L'Inde émerge comme puissance régionale avec des ambitions globales. Le Brésil domine en Amérique du Sud.""",
            "theme": "puissances_mondiales"
        },
        {
            "titre": "Les conflits régionaux actuels",
            "contenu": """Les tensions en mer de Chine méridionale impliquent plusieurs pays asiatiques. 
            Le conflit ukrainien reste central en Europe. Au Moyen-Orient, les questions israélo-palestiniennes perdurent. 
            En Afrique, le Sahel fait face à des défis sécuritaires majeurs. Les tensions frontalières persistent en Asie du Sud.""",
            "theme": "conflits"
        },
        {
            "titre": "Enjeux climatiques et géopolitique",
            "contenu": """Le changement climatique redéfinit les relations internationales. L'Arctique devient un nouvel espace de compétition. 
            Les pays insulaires militent pour des actions urgentes. La transition énergétique crée de nouvelles dépendances. 
            L'accès à l'eau devient un enjeu sécuritaire majeur dans plusieurs régions.""",
            "theme": "climat"
        },
        {
            "titre": "Organisations internationales et multilatéralisme",
            "contenu": """L'ONU cherche à se réformer face aux nouveaux défis. L'OTAN s'adapte aux menaces hybrides. 
            Les BRICS proposent des alternatives au système occidental. L'Union africaine gagne en influence. 
            Le G20 devient le forum principal de gouvernance économique mondiale.""",
            "theme": "organisations"
        },
        {
            "titre": "Technologies et souveraineté numérique",
            "contenu": """La 5G et l'IA deviennent des enjeux de souveraineté. Les États-Unis et la Chine rivalisent dans les semiconducteurs. 
            L'Europe développe sa stratégie numérique autonome. La cybersécurité est au cœur des préoccupations. 
            Les données sont le nouveau pétrole de l'économie mondiale.""",
            "theme": "technologie"
        }
    ]
    
    ref_folder = os.path.join(DOCS_FOLDER, "documents_reference")
    if not os.path.exists(ref_folder):
        os.makedirs(ref_folder)
    
    for i, doc in enumerate(documents_reference):
        filename = os.path.join(ref_folder, f"ref_{doc['theme']}.txt")
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"TITRE: {doc['titre']}\n")
            f.write(f"THEME: {doc['theme']}\n")
            f.write(f"\nCONTENU:\n{doc['contenu']}\n")
    
    print(f"\n✓ {len(documents_reference)} documents de référence créés dans {ref_folder}")

def main():
    """Fonction principale"""
    print("=== Récupération de documents géopolitiques ===")
    print(f"Dossier de destination: {DOCS_FOLDER}\n")
    
    tous_articles = []
    
    # Récupérer les articles RSS
    for source_key, source_info in SOURCES_RSS.items():
        articles = recuperer_articles_rss(source_key, source_info, max_articles=100)
        tous_articles.extend(articles)
        time.sleep(1)  # Politesse envers les serveurs
    
    # Sauvegarder les documents
    if tous_articles:
        sauvegarder_documents(tous_articles)
    
    # Générer des documents de référence
    generer_corpus_thematique()
    
    print(f"\n=== Récupération terminée ===")
    print(f"Total: {len(tous_articles)} articles d'actualité récupérés")
    print(f"Les documents sont prêts pour le TP RAG!")

if __name__ == "__main__":
    main()

=== Récupération de documents géopolitiques ===
Dossier de destination: documents_geopolitique


Récupération depuis Le Monde Diplomatique...
  ✓ Article 1: Polar et métaphysique...
  ✓ Article 2: Vers un nouvel ordre régional au Proche-Orient...
  ✓ Article 3: Tous influenceurs !...
  ✓ Article 4: Du judéo-bolchevisme culturel...
  ✓ Article 5: Des femmes très indociles...
  ✓ Article 6: De Constantinople à Istanbul...
  ✓ Article 7: Une tribu d'indésirables...
  ✓ Article 8: « Lycée pro », la dèche et le mépris...
  ✓ Article 9: La Crimée veut croire à la paix...
  ✓ Article 10: Papiers d'oranges, un monde en soie ?...
  ✓ Article 11: Le socialisme polonais survit dans l'assiette...
  ✓ Article 12: Gaza ou la faillite de l'Occident...
  ✓ Article 13: La monnaie n'est pas qu'une affaire d'hommes...
  ✓ Article 14: Culture club...
  ✓ Article 15: Au Cachemire, en attendant la prochaine fois...
  ✓ Article 16: Le parti des médias déjà en campagne...
  ✓ Article 17: Mythes et mensonges d

In [2]:
!pip install feedparser

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6089 sha256=70e446095be55e6fa2472ac11f9f726c0a74eaf964c6ec4c28476da1ce1ade28
  Stored in directory: /Users/raphaelcousin/Library/Caches/pip/wheels/3d/4d/ef/37cdccc18d6fd7e0dd7817dcdf9146d4d6789c32a227a28134
Successfully built sgmllib3k

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
